In [1]:
import pandas as pd

WSI = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_zestaw_1.2\22_09617A1_7.mrxs"
TILE = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_tiles_68x68_zestaw_1.2\train\Mieloblasty\22_09617A1_7_ROI_0_tile_0_mb.png"
ROI_DESC = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_ROI_zestaw_1.2\ROI_description.csv"
roi = {
    "roi_x_min": 49925.2188, 
    "roi_y_min": 120125.367, 
    "roi_w": 1531.3006999999998, 
    "roi_h": 1006.6169999999984
    }

In [2]:
roi_desc = pd.read_csv(ROI_DESC)
x = roi_desc[(roi_desc['slide'] == '22_09617A1_7') & (roi_desc['roi_id'] == 0)].iloc[0]
roi_desc.head()

,slide,roi_id,x_min,y_min,path,width,height
0,22_09617A1_7,0,49925.2188,120125.367,22_09617A1_7_ROI_0.png,1531.3007,1006.617
1,22_09617A1_7,1,52428.1836,129263.328,22_09617A1_7_ROI_1.png,1238.2539,926.375
2,22_09617A1_7,2,45226.6289,126018.805,22_09617A1_7_ROI_2.png,1802.5664,1487.047
3,22_09617A1_7,3,50775.1055,133331.062,22_09617A1_7_ROI_3.png,1677.5781,1081.704
4,22_09617A1_7,4,51859.1328,124660.156,22_09617A1_7_ROI_4.png,1528.3906,1381.422


In [3]:
OPENSLIDE_PATH = "C:\\Users\\MS\\openslide-bin-4.0.0.8-windows-x64\\bin"

import os

if hasattr(os, "add_dll_directory"):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
    print(openslide.__version__)
else:
    print("nope")
    import openslide
import numpy as np
import cv2

slide = openslide.OpenSlide(WSI)

roi_x_min=49925.2188
roi_y_min=120125.367 
roi_w=1531.3006999999998 
roi_h=1006.6169999999984

roi_pil = slide.read_region(
    (int(roi_x_min), int(roi_y_min)),  
    level=0,                 
    size=(int(roi_w), int(roi_h))      
).convert("RGB")

props = slide.properties
mpp_x = float(props.get("openslide.mpp-x", props.get("aperio.MPP", 0)))
mpp_y = float(props.get("openslide.mpp-y", props.get("aperio.MPP", 0)))

print(mpp_x, mpp_y)

1.3.1
0.242797397769517 0.242797397769517


In [4]:
from slide_tiling.tile_coordinates import get_tile_coordinates

coords = get_tile_coordinates(tile_filename=TILE, slide_filename=WSI, **roi)
print("Coordinates: ", coords)

1.3.1
Coordinates:  (50230.2188, 120242.367)


In [ ]:
from tqdm import tqdm

filename = "cd34_class_probabilities"
PROB_CSV = rf"..\data\adding_coords\{filename}_yolo.xlsx"
PROB_XLSX = rf"..\data\adding_coords\{filename}_yolo.xlsx"
DATA_DIR = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_zestaw_2.1"
TILE_DIR = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_tiles_yolo\wycinki_68x68\test\brazowe_komorki"
OUT_DIR = r"..\data\adding_coords"
ROI_DESC = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_ROI_zestaw_2.1\ROI_description.csv"
roi_desc = pd.read_csv(ROI_DESC)
df = pd.read_excel(PROB_XLSX)

out_columns = ["class", "predicted class", "Megakariocyty probability", "tile filename", "slide", "roi_id", "x", "y"]
df_out = pd.DataFrame(columns=out_columns)

df_out_data = {col: [] for col in out_columns}

class_map = {"Megakariocyty": 0, "Mieloblasty": 1}
reverse_class_map = {v: k for k, v in class_map.items()}

# "21_09830A1_19_ROI_0_0_3_cls1_4.png"
# "22_19269A1_7_ROI_0_tile_45_mc.png"
for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    tile_filename = row["filenames"].strip("b'")
    # print(tile_filename)
    # tile_filename = "21_09830A1_19_ROI_0_0_3_cls1_4"
    parts = tile_filename.split(".")[0].split("_")
    wsi = "_".join(parts[:3])
    roiid = parts[4]
    tileid = parts[6] if parts[5] == "tile" else tuple(int(parts[x]) for x in [5, 6, 8])
    
    if isinstance(tileid, tuple):
        classname = "Unknown"
        tile_pathname = f"{TILE_DIR}\{tile_filename}"
    else:
        classname = "Megakariocyty" if parts[7] == "mc" else "Mieloblasty"
        tile_pathname = f"{TILE_DIR}\{classname}\{tile_filename}"
    
    
    # print("WSI: ", wsi)
    # print("ROI: ", roiid)
    roi_info = roi_desc[(roi_desc['slide'] == wsi) & (roi_desc['roi_id'] == int(roiid))].iloc[0]
    
    wsi_pathname = f"{DATA_DIR}\{wsi}.mrxs"
    annotation_coords = get_tile_coordinates(
        tile_filename=tile_pathname, 
        slide_filename=wsi_pathname, 
        roi_x_min=roi_info["x_min"],
        roi_y_min=roi_info["y_min"],
        roi_w=roi_info["width"],
        roi_h=roi_info["height"],
        )
    
    df_out_data["roi_id"].append(roiid) 
    df_out_data["slide"].append(wsi)
    df_out_data["tile filename"].append(tile_filename)
    df_out_data["class"].append(classname)
    df_out_data["predicted class"].append(reverse_class_map[int(row["classes"])])
    df_out_data["Megakariocyty probability"].append(row["Megakariocyty probability"])
    df_out_data["x"].append(annotation_coords[0])
    df_out_data["y"].append(annotation_coords[1])


df_out = pd.DataFrame(data=df_out_data)

df_out.to_excel(f"{OUT_DIR}\cd34_annotation_coords_yolo.xlsx", index=False)
df_out

Processing rows: 100%|██████████| 630/630 [08:17<00:00,  1.27it/s]


,class,predicted class,Megakariocyty probability,tile filename,slide,roi_id,x,y
0,Unknown,Megakariocyty,0.700040,21_09830A1_19_ROI_0_0_3_cls1_4.png,21_09830A1_19,0,45951.1055,158948.1560
1,Unknown,Megakariocyty,0.703954,21_09830A1_19_ROI_0_0_3_cls1_5.png,21_09830A1_19,0,45882.1055,159000.1560
2,Unknown,Megakariocyty,0.693953,21_09830A1_19_ROI_0_0_3_cls1_6.png,21_09830A1_19,0,45909.1055,158979.1560
3,Unknown,Megakariocyty,0.547423,21_09830A1_19_ROI_0_10_13_cls1_10.png,21_09830A1_19,0,47210.1055,160217.1560
4,Unknown,Mieloblasty,0.362648,21_09830A1_19_ROI_0_10_13_cls1_12.png,21_09830A1_19,0,47198.1055,160228.1560
...,...,...,...,...,...,...,...,...
625,Unknown,Mieloblasty,0.248731,22_16277A1_7_ROI_1_4_1_cls1_19.png,22_16277A1_7,1,34004.3086,102939.2190
626,Unknown,Mieloblasty,0.254561,22_16277A1_7_ROI_1_4_2_cls1_9.png,22_16277A1_7,1,34101.3086,102879.2190
627,Unknown,Megakariocyty,0.596172,22_16278A1_7_ROI_0_0_2_cls1_39.png,22_16278A1_7,0,29191.7090,99419.4453
628,Unknown,Mieloblasty,0.465963,22_16278A1_7_ROI_0_2_3_cls1_21.png,22_16278A1_7,0,29283.7090,99632.4453


In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd

all_colour = "#F4FA58"
annotation_colours = {
    "komorki": "#ffff00",
    "Megakariocyty": "#ff0000",
    "Mieloblasty": "#0055ff",
    "Artefakty": "#ff00ff",
    "Naczynia": "#0055ff",
    "Obszary": "#ffaa00",
    "Unknown": "#46cc95",
}


def get_next_annotation_number(xml_root):
    existing_annotations = xml_root.findall("./Annotations/Annotation")

    if existing_annotations is None:
        return None
    if len(existing_annotations) < 1:
        return 0

    last_annotation_name = existing_annotations[-1].attrib.get("Name")
    next_annotation_number = int(last_annotation_name.split()[-1]) + 1
    return next_annotation_number


def add_annotation_groups(xml_root, new_group_names, annotation_colours):

    existing_groups = xml_root.findall(".AnnotationGroups/AnnotationGroup")
    existing_groups_root = xml_root.find(".//AnnotationGroups")
    
    new_group_names = set(new_group_names)
    if existing_groups is not None or len(existing_groups) > 0:
        existing_group_names = set(
            [group.attrib.get("Name") for group in existing_groups]
        )
        new_group_names = new_group_names.difference(existing_group_names)

    for group_name in new_group_names:
        new_group_attribs = {
            "Name": group_name,
            "PartOfGroup": "None",
            "Color": annotation_colours[group_name],
        }
        new_group = ET.Element("AnnotationGroup", attrib=new_group_attribs)
        existing_groups_root.append(new_group)


def add_annotations(xml_dir, out_dir, annotation_coords_file):
    
    df = pd.read_excel(annotation_coords_file)
    
    wsi_list = df_out["slide"].unique()
    
    annotation_names = []
    
    for wsi in tqdm(wsi_list):
        xml_pathname = f"{xml_dir}\{wsi}.xml"
        tree = ET.parse(xml_pathname)
        root = tree.getroot()
        
        existing_annotations = root.find(".//Annotations")
        
        annotation_info = df[df["slide"] == wsi]
        
        next_annotation_number = get_next_annotation_number(root)
        
        for i, row in annotation_info.iterrows():
            
            annotation_name = f"Annotation {next_annotation_number}"
        

            new_annotation_attrib = {
                "Name": annotation_name,
                "Type": "Dot",
                "PartOfGroup": row["predicted class"],
                "Color": "#F4FA58",
            }
            next_annotation_number += 1
            
            new_annotation = ET.Element("Annotation", attrib=new_annotation_attrib)
            new_coords = ET.Element("Coordinates")
            
            coord_attribs = {"Order": "0", "X": str(row["x"]), "Y": str(row["y"])}
            coord = ET.Element("Coordinate", attrib=coord_attribs)
            new_coords.append(coord)
            new_annotation.append(new_coords)
            existing_annotations.append(new_annotation)
            
        ET.indent(tree, space="  ")
        add_annotation_groups(root, ["Mieloblasty", "Megakariocyty"], annotation_colours=annotation_colours)
        out_file = Path(f"{out_dir}\{wsi}.xml")
        tree.write(out_file, encoding="utf-8", xml_declaration=True)
        
        df["annotation names"] = annotation_names
        df.to_excel(annotation_coords_file, index=False)
        
OUT = r"..\data\adding_coords\xmls"
FILE = r"..\data\adding_coords\cd34_annotation_coords_yolo.xlsx"
add_annotations(DATA_DIR, OUT, FILE)     

  0%|          | 0/9 [00:00<?, ?it/s]


ValueError: Length of values (263) does not match length of index (630)